## Agente Redutor Líquido de Nox Automotivo (ARLA 32) - PT Inmetro nº 139/2011/PT Inmetro nº 213/2021

In [ ]:
def scrapper():

    # IMPORT LIBRARIES
    from selenium import webdriver
    from selenium.webdriver.support import select
    from selenium.webdriver.support.select import Select
    from selenium.webdriver.support.ui import WebDriverWait
    from bs4 import BeautifulSoup
    import pandas as pd
    import time
    from time import sleep
    import re
    from tkinter import E
    import undetected_chromedriver.v2 as uc
    import scrapy
    

    # DEFINE VARIABLES
    print('Insira o nome do produto que vc quer buscar')
    finder_text  = input()
    
    
    
    def navegador():

        print('Abrindo o navegador, aguarde página carregar...')
        # GET BROWSER CHROME
        extset = ['enable-automation', 'ignore-certificate-errors']
        options = webdriver.ChromeOptions()
        options.add_argument("--start-maximized")


        navegador = uc.Chrome(version_main=106, options=options)

        # ENTER AT A LINK YOU PASSED
        navegador.get('http://www.inmetro.gov.br/prodcert/certificados/busca.asp')

        # FIND THE XPATH OF THE ELEMENT U WANT AND CLICK
        dropdown = navegador.find_element('xpath' ,'//*[@id="classe_produto"]')
        dd = Select(dropdown)

        # CHOOSE THE DROP DOWN MENU WHAT U WANT TO FIND
        dd.select_by_visible_text(finder_text)

        # CLICK TO SEARCH
        navegador.find_element('xpath', '/html/body/table[3]/tbody/tr[2]/td[2]/form/table[5]/tbody/tr/td[1]/input').click()

        navegador.implicitly_wait(10)

        return navegador


    def get_content(navegador):

        # VARIABLES
        item          = 2
        inmetro_dict  = dict()
        total_pag     = 0
        marca         ='a'
        modelo        = 0
        importado     = 0
        descricao     = 0
        pagina        = 2
        item          = 2
        tabela        = 1
        xpath_inmetro = '/html/body/form/table[3]/tbody/tr[2]/td[2]/table[2]/tbody/tr/td/table/tbody/tr/td/'


        # PARSE WITH BS4 LIB
        source = navegador.page_source
        soup = BeautifulSoup(source, 'lxml')
        paginas = soup.select('span', {'class': 'size10'})
        total_paginas = [i.text for i in paginas]
        total_pag = int([i for i in  total_paginas[-1] if re.compile('\d').match(i)][0])


        # TRY TO GET OTHER PRODUCT FROM SAME CNPJ
        print('Preparando para retirar o conteudo da página, aguarde...')
        while pagina != total_pag:
            
            xpath_inmetro = '/html/body/form/table[3]/tbody/tr[2]/td[2]/table[2]/tbody/tr/td/table/tbody/tr/td/'
            
            if tabela != 30:


                while (navegador.find_element('xpath', f'{xpath_inmetro}table[{tabela}]/tbody/tr[4]/td/table/tbody/tr[{item}]/td[1]').text) != ' ':
                    
                    # GET VARIABLES
                    # print('Iterando entre os elementos dentro da página')

                    # COPY HEADER TEXT FROM BODY PAGE
                    element = navegador.find_element('xpath', f"{xpath_inmetro}table[{tabela}]/tbody/tr[1]/td")

                    # GET NCHILD DIVS
                    nchild_elements_css = element.find_elements('css selector', '*')
                    nchild_elements_css = [i.text for i in nchild_elements_css]

                    # GET VARIABLES
                    certificador       = nchild_elements_css[2]
                    certificado        = nchild_elements_css[3]
                    tipo               = nchild_elements_css[4]
                    data_emissao       = nchild_elements_css[5]
                    validade           = nchild_elements_css[6]
                    status_certificado = nchild_elements_css[7]
                    cnpj               = navegador.find_element('xpath', f'{xpath_inmetro}table[{tabela}]/tbody/tr[3]/td[1]').text                                        
                    razao_social       = navegador.find_element('xpath', f'{xpath_inmetro}table[{tabela}]/tbody/tr[3]/td[2]').text
                    nome_fantasia      = navegador.find_element('xpath', f'{xpath_inmetro}table[{tabela}]/tbody/tr[3]/td[3]').text
                    endereco           = navegador.find_element('xpath', f'{xpath_inmetro}table[{tabela}]/tbody/tr[3]/td[4]').text
                    status             = navegador.find_element('xpath', f'{xpath_inmetro}table[{tabela}]/tbody/tr[3]/td[5]').text
                    papel_empresa      = navegador.find_element('xpath', f'{xpath_inmetro}table[{tabela}]/tbody/tr[3]/td[6]').text
                    marca              = navegador.find_element('xpath', f'{xpath_inmetro}table[{tabela}]/tbody/tr[4]/td/table/tbody/tr[{item}]/td[1]').text
                    modelo             = navegador.find_element('xpath', f'{xpath_inmetro}table[{tabela}]/tbody/tr[4]/td/table/tbody/tr[{item}]/td[2]').text
                    importado          = navegador.find_element('xpath', f'{xpath_inmetro}table[{tabela}]/tbody/tr[4]/td/table/tbody/tr[{item}]/td[3]').text
                    descricao          = navegador.find_element('xpath', f'{xpath_inmetro}table[{tabela}]/tbody/tr[4]/td/table/tbody/tr[{item}]/td[4]').text
                    
                  

                    # DICT WITH INFORMATION SCRAPPED
                    inmetro_dict2 = {
                            'certificador': [certificador], 'n_certificado': [certificado], 'tipo': [tipo], 'data_emissao': [data_emissao],\
                            'validade': [validade], 'status_certificado': [status_certificado], 'CNPJ': [cnpj],\
                            'razao_social': [razao_social], 'nome_fantasia': [nome_fantasia], 'endereco': [endereco], 'status': [status],\
                            'papel_empresa': [papel_empresa],'marca': [marca], 'modelo': [modelo], 'importado': [importado], 'descricao': [descricao]    
                                }


                    # PUT VALUES INTO DICT WITHOUT OVERWRITE
                    # print("Salvando os itens no dicionário")
                    for key, value in inmetro_dict2.items():
                        if key in inmetro_dict:
                            if isinstance(inmetro_dict[key], list):
                                inmetro_dict[key].append(value)
                            else:
                                temp_list = [inmetro_dict[key]]
                                temp_list.append(value)
                                inmetro_dict[key] = temp_list
                        else:
                            inmetro_dict[key] = value


                    # GO TO NEXT ITEM AT ROW ABOVE
                    item += 1
                    print(f'Marca atual: {marca}, item atual: {item}')
                    
                    # print(marca)
                    
                else:
                    
                    # print('trocando de item')
                    tabela += 1
                    item = 2

            else:
                # CHANGE PAGE
                print('troca de pagina')
                navegador.find_element('xpath', f'/html/body/form/table[3]/tbody/tr[2]/td[2]/table[3]/tbody/tr/td[2]/span[{pagina}]').click()

                sleep(20)
                
                '''
                #--- If you need to scroll down to load the page before scrapping
                the code above will do that for you ---#

                # SLEEP TIME TO WAIT PAGE LOAD
                time.sleep(5)


                SCROLL_PAUSE_TIME = 0.5

                # Get scroll height
                last_height = navegador.execute_script("return document.body.scrollHeight")

                while True:
                    # Scroll down to bottom
                    navegador.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                    # Wait to load page 
                    time.sleep(SCROLL_PAUSE_TIME)

                    # Calculate new scroll height and compare with last scroll height
                    new_height = navegador.execute_script("return document.body.scrollHeight")
                    if new_height == last_height:
                        break
                    last_height = new_height
                '''
                

                pagina += 1


        else:
            # BROWSER CLOSE
            navegador.close()


        try:                           
            # TRANSFORM DICT TO DF AND TO CSV
            inmetro_dict_df = pd.DataFrame.from_dict(inmetro_dict)
            inmetro_dict_df = pd.DataFrame.to_csv(inmetro_dict_df)

            # SAVING
            print('Digite o nome que vc quer salvar o arquivo: ')
            file_name = input()
            with open(f'C:/{file_name}.csv', 'wb') as f:
                f.write(inmetro_dict_df.encode('latin1'))
                f.close()
                print('Arquivo gerado com sucesso.')
        except:
            print('Não foi possivel gerar o arquivo.')
            return
            

        return 
        
    return get_content(navegador())


scrapper()


In [ ]:
Agente Redutor Líquido de Nox Automotivo (ARLA 32) - PT Inmetro nº 139/2011/PT Inmetro nº 213/2021